## Extracting the features from wav files

In [2]:
# importing required packages
import librosa
import pandas as pd
import numpy as np
import os
from pydub import AudioSegment
from tqdm import tqdm
from IPython.display import Audio

import warnings
warnings.filterwarnings('ignore')

In [3]:
# function to calculate the mean and variance
def mean_var_calculator(x_list):
    n_array = np.array(x_list)
    return list(n_array.mean(axis=1)), list(n_array.var(axis=1))

In [4]:
# function to extract the features from wav file
def features(song_id,category,path):

    x , sr = librosa.load(path)
    tempo = librosa.beat.tempo(x, sr = sr)
    zero_crossings = sum(librosa.zero_crossings(x, pad=False))
    
    n_sb = librosa.util.normalize(librosa.feature.spectral_bandwidth(x, sr=sr)[0])
    spectral_bandwidth_mean=n_sb.mean()
    spectral_bandwidth_var=n_sb.var()
        
    n_sc = librosa.util.normalize(librosa.feature.spectral_contrast(x, sr=sr)[0])
    spectral_contrast_mean=n_sc.mean()
    spectral_contrast_var=n_sc.var()
    
    n_scc = librosa.util.normalize(librosa.feature.spectral_centroid(x, sr=sr)[0])
    spectral_centroids_mean=n_scc.mean()
    spectral_centroids_var=n_scc.var()
        
    n_sr = librosa.util.normalize(librosa.feature.spectral_rolloff(x, sr=sr)[0])
    spectral_rolloff_mean=n_sr.mean()
    spectral_rolloff_var=n_sr.var()
    
    n_sf = librosa.util.normalize(librosa.feature.spectral_flatness(x)[0])
    spectral_flatness_mean=n_sf.mean()
    spectral_flatness_var=n_sf.var()
    
    chroma_stft_mean, chroma_stft_var  = mean_var_calculator(librosa.feature.chroma_stft(x, sr = sr))
    chroma_cqt_mean, chroma_cqt_var = mean_var_calculator(librosa.feature.chroma_cqt(x, sr = sr))
    chroma_cens_mean, chroma_cens_var = mean_var_calculator(librosa.feature.chroma_cens(x, sr = sr))
    #melspectrogram_mean, melspectrogram_var = mean_var_calculator(librosa.feature.melspectrogram(x, sr = sr))
    mfcc_mean, mfcc_var = mean_var_calculator(librosa.feature.mfcc(x, sr = sr))
    
    features = [song_id, tempo[0], zero_crossings, spectral_bandwidth_mean, spectral_bandwidth_var, spectral_contrast_mean, spectral_contrast_var,
            spectral_centroids_mean, spectral_centroids_var, spectral_rolloff_mean, spectral_rolloff_var, spectral_flatness_mean, spectral_flatness_var ]
    features.extend(chroma_stft_mean)
    features.extend(chroma_stft_var)
    features.extend(chroma_cqt_mean)
    features.extend(chroma_cqt_var)
    features.extend(chroma_cens_mean)
    features.extend(chroma_cens_var)
    features.extend(mfcc_mean)
    features.extend(mfcc_var)
    features.append(category)
    
    return features

In [5]:
columns=['song_id','tempo','zero_crossings','spectral_bandwidth_mean','spectral_bandwidth_var','spectral_contrast_mean','spectral_contrast_var',
         'spectral_centroids_mean','spectral_centroids_var','spectral_rolloff_mean','spectral_rolloff_var','spectral_flatness_mean','spectral_flatness_var',
         
         'chroma_stft1_mean', 'chroma_stft2_mean', 'chroma_stft3_mean', 'chroma_stft4_mean', 'chroma_stft5_mean', 'chroma_stft6_mean', 'chroma_stft7_mean', 
         'chroma_stft8_mean', 'chroma_stft9_mean', 'chroma_stft10_mean', 'chroma_stft11_mean', 'chroma_stft12_mean', 'chroma_stft1_var', 'chroma_stft2_var', 
         'chroma_stft3_var', 'chroma_stft4_var', 'chroma_stft5_var', 'chroma_stft6_var', 'chroma_stft7_var', 'chroma_stft8_var', 'chroma_stft9_var', 
         'chroma_stft10_var', 'chroma_stft11_var', 'chroma_stft12_var',
         
         'chroma_cqt1_mean', 'chroma_cqt2_mean', 'chroma_cqt3_mean', 'chroma_cqt4_mean', 'chroma_cqt5_mean', 'chroma_cqt6_mean', 'chroma_cqt7_mean', 
         'chroma_cqt8_mean', 'chroma_cqt9_mean', 'chroma_cqt10_mean', 'chroma_cqt11_mean', 'chroma_cqt12_mean', 'chroma_cqt1_var', 'chroma_cqt2_var', 
         'chroma_cqt3_var', 'chroma_cqt4_var', 'chroma_cqt5_var', 'chroma_cqt6_var', 'chroma_cqt7_var', 'chroma_cqt8_var', 'chroma_cqt9_var', 
         'chroma_cqt10_var', 'chroma_cqt11_var', 'chroma_cqt12_var',
         
         'chroma_cens1_mean', 'chroma_cens2_mean', 'chroma_cens3_mean', 'chroma_cens4_mean', 'chroma_cens5_mean', 'chroma_cens6_mean', 'chroma_cens7_mean', 
         'chroma_cens8_mean', 'chroma_cens9_mean', 'chroma_cens10_mean', 'chroma_cens11_mean', 'chroma_cens12_mean','chroma_cens1_var', 'chroma_cens2_var', 
         'chroma_cens3_var', 'chroma_cens4_var', 'chroma_cens5_var', 'chroma_cens6_var', 'chroma_cens7_var', 'chroma_cens8_var', 'chroma_cens9_var', 
         'chroma_cens10_var', 'chroma_cens11_var', 'chroma_cens12_var',
         
         'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean', 'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean', 'mfcc9_mean', 'mfcc10_mean', 
         'mfcc11_mean', 'mfcc12_mean', 'mfcc13_mean', 'mfcc14_mean', 'mfcc15_mean', 'mfcc16_mean', 'mfcc17_mean', 'mfcc18_mean', 'mfcc19_mean', 
         'mfcc20_mean', 'mfcc1_var', 'mfcc2_var', 'mfcc3_var', 'mfcc4_var', 'mfcc5_var', 'mfcc6_var', 'mfcc7_var', 'mfcc8_var', 'mfcc9_var',
         'mfcc10_var', 'mfcc11_var', 'mfcc12_var', 'mfcc13_var', 'mfcc14_var', 'mfcc15_var', 'mfcc16_var', 'mfcc17_var', 'mfcc18_var', 
         'mfcc19_var', 'mfcc20_var', 'Artist_name']

len(columns)

126

### Extracting features for English Songs

#### Extracting artist data features from vocals data for collected deezer data

In [5]:
# English songs Vocal_features Extraction
source = r'Deezer Data\artist\wav'
dest = r'Deezer Data\artist\splitter'
songs = os.listdir(source)
print('Total number of songs:', len(songs))
songs_features = []

Total number of songs: 189


In [6]:
# English songs Vocal_features Extraction
for song in tqdm(songs):
    song_id = song[:song.find('.')]
    artist_name = song[:song.find('_')]
    path = dest + '\\' + song_id + '\\vocals.wav'
    audio = AudioSegment.from_mp3(path)
    
    for i in range(round(len(audio)/5000)):
        try:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)
        except:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output2.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)


100%|██████████| 189/189 [48:30<00:00, 15.40s/it]


In [7]:
df = pd.DataFrame(songs_features, columns = columns)
df.shape

(1134, 126)

In [9]:
df.to_csv(r'Deezer Data\artist\English_artist_data_vocal_features_deezer.csv', index=False)

#### Extracting artist data features from vocals data for collected JioSaven data

In [ ]:
# English songs Vocal_features Extraction
source = r'JioSavan Data\English\artist\wav'
dest = r'JioSavan Data\English\artist\splitter'
songs = os.listdir(source)
print('Total number of songs:', len(songs))
songs_features = []

In [ ]:
# English songs Vocal_features Extraction
for song in tqdm(songs):
    song_id = song[:song.find('.')]
    artist_name = song[:song.find('_')]
    path = dest + '\\' + song_id + '\\vocals.wav'
    audio = AudioSegment.from_mp3(path)
    
    for i in range(round(len(audio)/5000)):
        try:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)
        except:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output2.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)


In [ ]:
df = pd.DataFrame(songs_features, columns = columns)
df.to_csv(r'JioSavan Data\English\artist\English_artist_data_vocal_features.csv', index=False)

#### Extracting genres data features from the collected JioSaven data

In [ ]:
# English songs features Extraction genre
source = r'JioSavan Data\English\genre\wav'
songs = os.listdir(source)
print('Total number of songs:', len(songs))
songs_features = []

In [ ]:
# English songs Vocal_features Extraction
for song in tqdm(songs):
    song_id = song[:song.find('.')]
    artist_name = song[:song.find('_')]
    path = source + '\\' + song
    audio = AudioSegment.from_mp3(path)
    
    for i in range(round(len(audio)/5000)):
        try:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)
        except:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output2.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)


In [ ]:
df = pd.DataFrame(songs_features, columns = columns)
df.to_csv(r'JioSavan Data\English\genre\English_genres_data_features.csv', index=False)

## Extracting features for Telugu lanuage
### Features extraction for genres data

In [ ]:
# Telugu songs features Extraction genre
source = r'JioSavan Data\Telugu\genre\wav'
songs = os.listdir(source)
print('Total number of songs:', len(songs))
songs_features = []

In [ ]:
# Telugu songs Vocal_features Extraction
for song in tqdm(songs):
    song_id = song[:song.find('.')]
    artist_name = song[:song.find('_')]
    path = source + '\\' + song
    audio = AudioSegment.from_mp3(path)
    
    for i in range(round(len(audio)/5000)):
        try:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)
        except:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output2.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)


In [ ]:
df = pd.DataFrame(songs_features, columns = columns)
df.to_csv(r'JioSavan Data\Telugu\genre\Telugu_genres_data_features.csv', index=False)

### Features extraction for Artists data

In [ ]:
# Telugu songs Vocal_features Extraction
source = r'JioSavan Data\Telugu\artist\wav'
dest = r'JioSavan Data\Telugu\artist\splitter'
songs = os.listdir(source)
print('Total number of songs:', len(songs))
songs_features = []

In [ ]:
# Telugu songs Vocal_features Extraction
for song in tqdm(songs):
    song_id = song[:song.find('.')]
    artist_name = song[:song.find('_')]
    path = dest + '\\' + song_id + '\\vocals.wav'
    audio = AudioSegment.from_mp3(path)
    
    for i in range(round(len(audio)/5000)):
        try:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)
        except:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output2.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)

In [ ]:
df = pd.DataFrame(songs_features, columns = columns)
df.to_csv(r'JioSavan Data\Telugu\artist\Telugu_artist_data_vocal_features.csv', index=False)

## Extracting features for Hindi lanuage
### Features extraction for genres data

In [ ]:
# Hindi songs features Extraction genre
source = r'JioSavan Data\Hindi\genre\wav'
songs = os.listdir(source)
print('Total number of songs:', len(songs))
songs_features = []

In [ ]:
# Hindi songs Vocal_features Extraction
for song in tqdm(songs):
    song_id = song[:song.find('.')]
    artist_name = song[:song.find('_')]
    path = source + '\\' + song
    audio = AudioSegment.from_mp3(path)
    
    for i in range(round(len(audio)/5000)):
        try:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)
        except:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output2.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)


In [ ]:
df = pd.DataFrame(songs_features, columns = columns)
df.to_csv(r'JioSavan Data\Hindi\genre\Hindi_genres_data_features.csv', index=False)

### Features extraction for Artists data

In [ ]:
# Telugu songs Vocal_features Extraction
source = r'JioSavan Data\Hindi\artist\wav'
dest = r'JioSavan Data\Hindi\artist\splitter'
songs = os.listdir(source)
print('Total number of songs:', len(songs))
songs_features = []

In [ ]:
# Telugu songs Vocal_features Extraction
for song in tqdm(songs):
    song_id = song[:song.find('.')]
    artist_name = song[:song.find('_')]
    path = dest + '\\' + song_id + '\\vocals.wav'
    audio = AudioSegment.from_mp3(path)
    
    for i in range(round(len(audio)/5000)):
        try:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)
        except:
            song = audio[i*5000:(i+1)*5000] 
            out_path = 'output2.wav'
            song.export(out_path, format="wav")
            temp = features(song_id + '_' + str(i), artist_name , out_path)
            songs_features.append(temp)
            os.remove(out_path)

In [ ]:
df = pd.DataFrame(songs_features, columns = columns)
df.to_csv(r'JioSavan Data\Hindi\artist\Hindi_artist_data_vocal_features.csv', index=False)